In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import itertools
import pickle

In [5]:
df = pd.read_csv("../data/run-over-dataset.csv")

columns_to_mantain = ['SESSO', 'ANNI', 'ALTEZZA', 'PESO', 'BMI', 'Mezzo', 'Tot Testa', 'Tot Torace', 'Tot Addome', 
                    'Tot Scheletro', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx', 'Omero dx', 'Radio dx', 'Ulna dx',
                    'Omero sx', 'Radio sx', 'Ulna sx', 'carpo dx', 'I metacarpo dx', 'II metacarpo dx', 'III metacarpo dx',
                    'IV  metacarpo dx', 'V metacarpo dx', 'I raggio dx', 'II raggio dx', 'III raggio dx',
                    'IV raggio dx', 'V raggio dx', 'carpo sx', 'I metacarpo sx', 'II metacarpo sx', 'III metacarpo sx',
                    'IV metacarpo sx', 'V metacarpo sx', 'I raggio sx', 'II raggio sx', 'III raggio sx', 'IV raggio sx',
                    'V raggio sx', 'Spalla dx', 'Spalla sx', 'Gomito dx', 'Gomito sx', 'Polso dx', 'Polso sx', 
                    'femore dx', 'tibia dx', 'fibula dx', 'rotula dx', 'femore sx', 'tibia sx', 'fibula sx', 
                    'rotula sx', 'tarso dx', 'I metatarso dx', 'II metatarso dx', 'III metatarso dx', 'IV metatarso dx',
                    'V metatarso dx', 'I raggio dx', 'II raggio dx', 'III raggio dx', 'IV raggio dx', 'V raggio dx', 
                    'tarso sx', 'I metatarso sx', 'II metatarso sx', 'III metatarso sx', 'IV metatarso sx', 
                    'V metatarso sx', 'I raggio sx', 'II raggio sx', 'III raggio sx', 'IV raggio sx', 'V raggio sx',
                    'Art. coxo-femorale dx', 'Art. coxo-femorale sx', 'Rotula o Ginocchio dx', 'Rotula o Ginocchio sx',
                    'Caviglia dx', 'Caviglia sx']

X = df.loc[:, columns_to_mantain]

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

print(X.shape)

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 115)
(130, 74)


In [8]:
def nested_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [9]:
results = nested_cv_accuracy(X)
results

{'algorythm': 'OneClassSVM',
 'best kernel': 'linear',
 'best gamma': 1e-09,
 'best nu': 0.61,
 'method used for model selection': 'accuracy on nested cv',
 'accuracy mean': 55.692307692307686,
 'accuracy std': 8.185714211952632,
 'precision mean': 67.83607503607502,
 'precision  std': 26.121421634982294,
 'recall mean': 29.28571428571429,
 'recall std': 16.24116714414582,
 'data set': 'tot columns, standard scaled'}

In [10]:
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"


In [11]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [12]:
def nested_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [13]:
scores_df = add_record(scores_df, nested_cv_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"


In [14]:
def outer_cv_inner_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [15]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.96,accuracy on outer cv inner holdout,50.692308,7.445661,46.478716,37.824486,17.571429,17.682575,"tot columns, standard scaled"


In [16]:
def outer_cv_inner_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [17]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.96,accuracy on outer cv inner holdout,50.692308,7.445661,46.478716,37.824486,17.571429,17.682575,"tot columns, standard scaled"
3,OneClassSVM,rbf,1.000000e-05,0.55,f1 on outer cv inner holdout,55.153846,10.366366,59.737941,22.022983,44.714286,25.022847,"tot columns, standard scaled"


In [18]:
def outer_holdout_inner_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [19]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.96,accuracy on outer cv inner holdout,50.692308,7.445661,46.478716,37.824486,17.571429,17.682575,"tot columns, standard scaled"
3,OneClassSVM,rbf,1.000000e-05,0.55,f1 on outer cv inner holdout,55.153846,10.366366,59.737941,22.022983,44.714286,25.022847,"tot columns, standard scaled"
4,OneClassSVM,linear,1.000000e-09,0.84,accuracy on outer holdout inner cv,49.230769,7.653750,47.166667,42.229860,12.308143,14.404871,"tot columns, standard scaled"


In [20]:
def outer_holdout_inner_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [21]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.96,accuracy on outer cv inner holdout,50.692308,7.445661,46.478716,37.824486,17.571429,17.682575,"tot columns, standard scaled"
3,OneClassSVM,rbf,1.000000e-05,0.55,f1 on outer cv inner holdout,55.153846,10.366366,59.737941,22.022983,44.714286,25.022847,"tot columns, standard scaled"
4,OneClassSVM,linear,1.000000e-09,0.84,accuracy on outer holdout inner cv,49.230769,7.653750,47.166667,42.229860,12.308143,14.404871,"tot columns, standard scaled"
5,OneClassSVM,linear,1.000000e-09,0.45,f1 on outer holdout inner cv,53.076923,10.714145,48.085498,32.638520,27.137030,18.794661,"tot columns, standard scaled"


In [22]:
def double_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [23]:
scores_df = add_record(scores_df, double_holdout_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.96,accuracy on outer cv inner holdout,50.692308,7.445661,46.478716,37.824486,17.571429,17.682575,"tot columns, standard scaled"
3,OneClassSVM,rbf,1.000000e-05,0.55,f1 on outer cv inner holdout,55.153846,10.366366,59.737941,22.022983,44.714286,25.022847,"tot columns, standard scaled"
4,OneClassSVM,linear,1.000000e-09,0.84,accuracy on outer holdout inner cv,49.230769,7.653750,47.166667,42.229860,12.308143,14.404871,"tot columns, standard scaled"
5,OneClassSVM,linear,1.000000e-09,0.45,f1 on outer holdout inner cv,53.076923,10.714145,48.085498,32.638520,27.137030,18.794661,"tot columns, standard scaled"
6,OneClassSVM,linear,1.000000e-09,0.50,accuracy on double holdout,50.000000,8.426501,67.960784,29.766643,22.547439,15.623310,"tot columns, standard scaled"


In [24]:
def double_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'tot columns, standard scaled'}

In [25]:
scores_df = add_record(scores_df, double_holdout_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,linear,1.000000e-09,0.61,accuracy on nested cv,55.692308,8.185714,67.836075,26.121422,29.285714,16.241167,"tot columns, standard scaled"
1,OneClassSVM,rbf,1.000000e-08,0.65,f1 on nested cv,54.461538,10.472845,53.841341,22.769196,45.571429,27.025687,"tot columns, standard scaled"
2,OneClassSVM,linear,1.000000e-09,0.96,accuracy on outer cv inner holdout,50.692308,7.445661,46.478716,37.824486,17.571429,17.682575,"tot columns, standard scaled"
3,OneClassSVM,rbf,1.000000e-05,0.55,f1 on outer cv inner holdout,55.153846,10.366366,59.737941,22.022983,44.714286,25.022847,"tot columns, standard scaled"
4,OneClassSVM,linear,1.000000e-09,0.84,accuracy on outer holdout inner cv,49.230769,7.653750,47.166667,42.229860,12.308143,14.404871,"tot columns, standard scaled"
5,OneClassSVM,linear,1.000000e-09,0.45,f1 on outer holdout inner cv,53.076923,10.714145,48.085498,32.638520,27.137030,18.794661,"tot columns, standard scaled"
6,OneClassSVM,linear,1.000000e-09,0.50,accuracy on double holdout,50.000000,8.426501,67.960784,29.766643,22.547439,15.623310,"tot columns, standard scaled"
7,OneClassSVM,linear,1.000000e-09,0.22,f1 on double holdout,52.307692,8.804249,49.508478,28.787761,31.820574,21.605680,"tot columns, standard scaled"


In [26]:
file_path = 'svm_exp4_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)